In [ ]:
import fastjet
import pandas as pd
import numpy as np
import awkward as ak
import vector
import matplotlib.pyplot as plt
import h5py

vector.register_awkward()

JET_ALGO = fastjet.antikt_algorithm
R = 1.0
MIN_JET_PT = 30.0
MAX_JETS = 2
MAX_PARTICLES_PER_JET = 10

chunks = [
    ("events_anomalydetection_v2.h5", 0, 100_000),  # background
    ("events_anomalydetection_v2.h5", 100_000, 200_000),  # background
    ("events_anomalydetection_v2.h5", 200_000, 300_000),  # background
    ("events_anomalydetection_v2.h5", 300_000, 400_000),  # background
    ("events_anomalydetection_v2.h5", 400_000, 500_000),  # background
    ("events_anomalydetection_v2.h5", 500_000, 600_000),  # background
    ("events_anomalydetection_v2.h5", 600_000, 700_000),  # background
    ("events_anomalydetection_v2.h5", 700_000, 800_000),  # background
    ("events_anomalydetection_v2.h5", 800_000, 900_000),  # background
    ("events_anomalydetection_v2.h5", 900_000, 1_000_000),  # background
    ("events_anomalydetection_v2.h5", 1_000_000, 1_100_000),  # two prong signal
    ("events_anomalydetection_Z_XY_qqq.h5", 0, 100_000),  # three prong signal
]

In [ ]:
# define helper function to make awkward arrays of fixed size
def fill_pad(ak_array, max_n=10, axis=-1, pad=0, clip=True):
    return ak.fill_none(
        ak.pad_none(ak_array, max_n, clip=clip, axis=axis), pad, axis=axis
    )


# given a dataframe, get formatted datasets
def get_datasets(df, filename):
    # reshape to 700 particles with 3 features (px, py, pz)
    p = df.values[:, :-1].reshape(len(df), 700, 3)

    # get labels
    labels = df.values[:, -1]

    # use label=0 for background, label=1 for two prong signal, label=2 for three prong signal
    if "events_anomalydetection_Z_XY_qqq" in filename:
        labels += 1.0

    # split into momentum components
    pt = p[..., 0]
    eta = p[..., 1]
    phi = p[..., 2]
    # mask out zero-padded particles
    mask = pt > 0

    # convert to awkard arrays
    ak_pt = ak.from_iter([a[m] for a, m in zip(pt, mask)])
    ak_eta = ak.from_iter([a[m] for a, m in zip(eta, mask)])
    ak_phi = ak.from_iter([a[m] for a, m in zip(phi, mask)])

    # use vector library to make a high-level array of 4-momenta
    # assuming massless particles
    particles = ak.zip(
        {
            "px": ak_pt * np.cos(ak_phi),
            "py": ak_pt * np.sin(ak_phi),
            "pz": ak_pt * np.sinh(ak_eta),
            "E": ak_pt * np.cosh(ak_eta),
        },
        with_name="MomentumArray4D",
    )

    # define jet clustering algorithm
    jetdef = fastjet.JetDefinition(JET_ALGO, R)

    # perform jet clustering; this can take a while (~2 minutes for 100k events on my MacBook)
    cluster = fastjet.ClusterSequence(particles, jetdef)

    # retrieve jets
    jets = cluster.inclusive_jets(min_pt=MIN_JET_PT)

    # sort jets by pt
    sorted_by_jet_pt = ak.argsort(jets.pt, ascending=False, axis=-1)
    jets = jets[sorted_by_jet_pt]

    # similarly sort constituents of jets by the jet pt
    constituents = cluster.constituents(min_pt=MIN_JET_PT)
    constituents = constituents[sorted_by_jet_pt]

    print(constituents)
    # within each jet, sort consituents by pt
    sorted_by_constituent_pt = ak.argsort(
        np.sqrt(constituents.px**2 + constituents.py**2), ascending=False, axis=-1
    )
    print(sorted_by_constituent_pt)
    constituents = constituents[sorted_by_constituent_pt]
    print(constituents)
    # zero-pad constituents to fixed size (MAX_JETS jets per event, MAX_PARTICLES_PER_JET particles per jet)
    constituents_padded_px = fill_pad(
        fill_pad(constituents.px, max_n=MAX_JETS, axis=-2, pad=[0]),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()
    constituents_padded_py = fill_pad(
        fill_pad(constituents.py, max_n=MAX_JETS, axis=-2, pad=[0]),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()
    constituents_padded_pz = fill_pad(
        fill_pad(constituents.pz, max_n=MAX_JETS, axis=-2, pad=[0]),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()
    constituents_pt = np.sqrt(constituents.px**2 + constituents.py**2)
    constituents_padded_pt = fill_pad(
        fill_pad(constituents_pt, max_n=MAX_JETS, axis=-2, pad=[0]),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()
    constituents_padded_eta = fill_pad(
        fill_pad(
            np.arcsinh(constituents.pz / constituents_pt),
            max_n=MAX_JETS,
            axis=-2,
            pad=[0],
        ),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()
    constituents_padded_phi = fill_pad(
        fill_pad(
            np.arctan2(constituents.py, constituents.px),
            max_n=MAX_JETS,
            axis=-2,
            pad=[0],
        ),
        max_n=MAX_PARTICLES_PER_JET,
        axis=-1,
        pad=0,
    ).to_numpy()

    # stack constituent px, py, pz, pt, eta, phi into a single array
    constituents_padded = np.stack(
        [
            constituents_padded_px,
            constituents_padded_py,
            constituents_padded_pz,
            constituents_padded_pt,
            constituents_padded_eta,
            constituents_padded_phi,
        ],
        axis=-1,
    )

    jets_padded_px = fill_pad(jets.px, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()
    jets_padded_py = fill_pad(jets.py, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()
    jets_padded_pz = fill_pad(jets.pz, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()

    jets_padded_pt = fill_pad(jets.pt, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()
    jets_padded_eta = fill_pad(jets.eta, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()
    jets_padded_phi = fill_pad(jets.phi, max_n=MAX_JETS, axis=-1, pad=0).to_numpy()

    jets_padded = np.stack(
        [
            jets_padded_px,
            jets_padded_py,
            jets_padded_pz,
            jets_padded_pt,
            jets_padded_eta,
            jets_padded_phi,
        ],
        axis=-1,
    )
    return constituents_padded, labels, jets_padded

In [ ]:
all_constituents_padded = []
all_labels = []
all_jets_padded = []

for i, (filename, start, stop) in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}: {filename}, {start}-{stop}")
    df = pd.read_hdf(filename, start=start, stop=stop)
    constituents_padded, labels, jets_padded = get_datasets(df, filename)

    print(f"constituents_padded={constituents_padded[0]}")
    print(f"labels={labels[0]}")
    print(f"jets_padded={jets_padded[0]}")
    all_constituents_padded.append(constituents_padded)
    all_labels.append(labels)
    all_jets_padded.append(jets_padded)

constituents_padded = np.concatenate(all_constituents_padded)
labels = np.concatenate(all_labels)
jets_padded = np.concatenate(all_jets_padded)

# write out hdf5 file
with h5py.File("constituents_rd_dataset.h5", "w") as output:
    output.create_dataset("constituents_padded", data=constituents_padded)
    output.create_dataset("labels", data=labels)
    output.create_dataset("jets_padded", data=jets_padded)

In [ ]:
import matplotlib.pyplot as plt

with h5py.File("constituents_rd_dataset.h5", "r") as input:
    print(input["constituents_padded"])
    print(input["labels"])
    print(input["jets_padded"])